In [2]:
from functions import *
from aug import *

import numpy as np
import pandas as pd
import pickle
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.manifold import TSNE



from keras import backend as K
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #get rid of warnings

2022-12-06 22:55:04.487433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 22:55:04.788119: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 22:55:05.430231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-06 22:55:05.430279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

<input type='checkbox'> Train model on Original + Augmented data (with different methods) and test it </input> 

In [ ]:
# load data
datasets = ['pc','cr','subj']
aug_methods = ['eda_augmenter','wordnet_augmenter','aeda_augmenter','backtranslation_augmenter','clare_augmenter']
n_samples = [1,2,4,8,10]
#path = f'data/{dataset}/train_{dataset}_{aug_method}_n_samples_{n_sample}.csv'


In [ ]:

def create_X_matrix(dataset, w2v,word2vec_len=300, batch_size=25):
    dataset_size = len(dataset)
    x_matrix = np.zeros((dataset_size, batch_size, word2vec_len))    
    for i, line in enumerate(dataset):
        words = line.split()
        words = words[:batch_size] #cut off if too long
        for j, word in enumerate(words):
            if word in w2v:
                x_matrix[i, j, :] = w2v[word]

    return x_matrix


def build_model(batch_size=25, word2vec_len=300):
    model = None
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(batch_size, word2vec_len)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #print(model.summary())
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    # print(f'Accuracy: {acc}')
    # print(f'F1: {f1}')
    # print(f'Precision: {precision}')
    # print(f'Recall: {recall}')
    return acc, f1, precision, recall


def run(dataset_name,aug_method,n_sample):
    # Load data
    path_train_original = f'data/{dataset_name}/train.txt'
    path_train_aug = f'data/{dataset_name}/train_{dataset_name}_{aug_method}_n_sample_{n_sample}.csv'
    path_test = f'data/{dataset_name}/test.txt'
    train_aug = load_data(path_train_aug)
    train_original = load_data(path_train_original)
    test_data = load_data(path_test)
    X_train_aug, y_train_aug = train_aug['text'].values, train_aug['class'].values
    X_train_original, y_train_original = train_original['text'].values, train_original['class'].values
    X_test, y_test = test_data['text'].values, test_data['class'].values

    # load wor2vec pickle
    path_w2v = f'data/{dataset_name}/word2vec.p'
    w2v = pickle.load(open(path_w2v, 'rb'))


    # create matrices
    X_train_aug = create_X_matrix(X_train_aug, w2v)
    X_train_original = create_X_matrix(X_train_original, w2v)
    X_test = create_X_matrix(X_test, w2v)

    # Train model
    model_aug = build_model()
    model_original = build_model()

    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

    model_aug.fit(X_train_aug, y_train_aug, epochs=1000,
                 callbacks=callbacks, validation_split=0.1,
                  batch_size=1024,shuffle=True, verbose=0)
    model_original.fit(X_train_original, y_train_original, epochs=1000,
                    callbacks=callbacks, validation_split=0.1,
                    batch_size=1024,shuffle=True, verbose=0)

    # Evaluate model
    acc_aug, f1_aug, precision_aug, recall_aug = evaluate_model(model_aug, X_test, y_test)
    acc_original, f1_original, precision_original, recall_original = evaluate_model(model_original, X_test, y_test)
    print(f'original model: \n acc: {acc_original} \n f1: {f1_original} \n precision: {precision_original} \n recall: {recall_original}')
    print(f'augmented model: \n acc: {acc_aug} \n f1: {f1_aug} \n precision: {precision_aug} \n recall: {recall_aug}')
    

    # Save model
    #model.save('model.h5')


In [ ]:
run('cr','eda_augmenter',4)

In [ ]:
df = load_data('data/cr/train_cr_eda_augmenter_n_sample_4.csv')

In [ ]:
X_train_aug, y_train_aug = df['text'].values, df['class'].values


In [ ]:
y_train_aug

In [ ]:
from keras.utils.np_utils import to_categorical
y_train=to_categorical(y_train_aug)

In [ ]:
y_train

In [ ]:
from functions import *
from aug import *

import numpy as np
import pandas as pd
import pickle
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.manifold import TSNE

from keras import backend as K
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #get rid of warnings


def create_X_matrix(dataset, w2v,word2vec_len=300, batch_size=25):
    dataset_size = len(dataset)
    x_matrix = np.zeros((dataset_size, batch_size, word2vec_len))    
    for i, line in enumerate(dataset):
        words = line.split()
        words = words[:batch_size] #cut off if too long
        for j, word in enumerate(words):
            if word in w2v:
                x_matrix[i, j, :] = w2v[word]

    return x_matrix

from keras.optimizers import RMSprop
def build_model(batch_size=25, word2vec_len=300):
    model = None
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(batch_size, word2vec_len)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))

    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))              
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['acc'] )
    #print(model.summary())
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    #y_pred = to_categorical(y_pred.argmax(axis=1))
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    # print(f'Accuracy: {acc}')
    # print(f'F1: {f1}')
    # print(f'Precision: {precision}')
    # print(f'Recall: {recall}')
    return acc, f1, precision, recall


def run(dataset_name,aug_method,n_sample):
    # Load data
    print('Loading data...')
    path_train_original = f'data/{dataset_name}/train.txt'
    path_train_aug = f'data/{dataset_name}/train_{dataset_name}_{aug_method}_n_sample_{n_sample}.csv'
    path_test = f'data/{dataset_name}/test.txt'
    train_aug = load_data(path_train_aug)
    train_original = load_data(path_train_original)
    test_data = load_data(path_test)
    X_train_aug, y_train_aug = train_aug['text'].values, train_aug['class'].values.astype(float)
    X_train_original, y_train_original = train_original['text'].values, train_original['class'].values.astype(float)
    X_test, y_test = test_data['text'].values, test_data['class'].values.astype(float)

    # y_train_aug = to_categorical(y_train_aug)
    # y_train_original = to_categorical(y_train_original)
    # y_test = to_categorical(y_test)


    # load wor2vec pickle
    path_w2v = f'data/{dataset_name}/word2vec.p'
    w2v = pickle.load(open(path_w2v, 'rb'))


    # create matrices
    print('Creating matrices...')
    X_train_aug = create_X_matrix(X_train_aug, w2v)
    X_train_original = create_X_matrix(X_train_original, w2v)
    X_test = create_X_matrix(X_test, w2v)

    # Train model
    print('Training model...')
    model_aug = build_model()
    model_original = build_model()

    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

    model_aug.fit(X_train_aug, y_train_aug, epochs=1000,
                 callbacks=callbacks, validation_split=0.1,
                  batch_size=1024,shuffle=True, verbose=0)
    model_original.fit(X_train_original, y_train_original, epochs=1000,
                    callbacks=callbacks, validation_split=0.1,
                    batch_size=1024,shuffle=True, verbose=0)

    # Evaluate model
    print('Evaluating model...')

    acc_aug, f1_aug, precision_aug, recall_aug = evaluate_model(model_aug, X_test, y_test)
    acc_original, f1_original, precision_original, recall_original = evaluate_model(model_original, X_test, y_test)
    print(f'original model: \n acc: {acc_original:.4f} \n f1: {f1_original:.4f} \n precision: {precision_original:.4f} \n recall: {recall_original:.4f}')
    print(f'augmented model: \n acc: {acc_aug:.4f} \n f1: {f1_aug:.4f} \n precision: {precision_aug:.4f} \n recall: {recall_aug:.4f}')

    print('finished')

# if __name__ == '__main__':

#     run('cr','eda_augmenter',4)


In [ ]:
dataset_name,aug_method,n_sample = 'cr','eda_augmenter',4

print('Loading data...')
path_train_original = f'data/{dataset_name}/train.txt'
path_train_aug = f'data/{dataset_name}/train_{dataset_name}_{aug_method}_n_sample_{n_sample}.csv'
path_test = f'data/{dataset_name}/test.txt'
train_aug = load_data(path_train_aug)
train_original = load_data(path_train_original)
test_data = load_data(path_test)
X_train_aug, y_train_aug = train_aug['text'].values, train_aug['class'].values.astype(float)
X_train_original, y_train_original = train_original['text'].values, train_original['class'].values.astype(float)
X_test, y_test = test_data['text'].values, test_data['class'].values.astype(float)

# y_train_aug = to_categorical(y_train_aug)
# y_train_original = to_categorical(y_train_original)
# y_test = to_categorical(y_test)


# load wor2vec pickle
path_w2v = f'data/{dataset_name}/word2vec.p'
w2v = pickle.load(open(path_w2v, 'rb'))


# create matrices
print('Creating matrices...')
X_train_aug = create_X_matrix(X_train_aug, w2v)
X_train_original = create_X_matrix(X_train_original, w2v)
X_test = create_X_matrix(X_test, w2v)

# Train model
print('Training model...')
model_aug = build_model()
model_original = build_model()

callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

model_aug.fit(X_train_aug, y_train_aug, epochs=1000,
                callbacks=callbacks, validation_split=0.1,
                batch_size=1024,shuffle=True, verbose=0)
model_original.fit(X_train_original, y_train_original, epochs=1000,
                callbacks=callbacks, validation_split=0.1,
                batch_size=1024,shuffle=True, verbose=0)

# Evaluate model
print('Evaluating model...')

# acc_aug, f1_aug, precision_aug, recall_aug = evaluate_model(model_aug, X_test, y_test)
# acc_original, f1_original, precision_original, recall_original = evaluate_model(model_original, X_test, y_test)
# print(f'original model: \n acc: {acc_original:.4f} \n f1: {f1_original:.4f} \n precision: {precision_original:.4f} \n recall: {recall_original:.4f}')
# print(f'augmented model: \n acc: {acc_aug:.4f} \n f1: {f1_aug:.4f} \n precision: {precision_aug:.4f} \n recall: {recall_aug:.4f}')

print('finished')

In [ ]:
pred = model_original.predict(X_test)
y = to_categorical(y_test)

In [ ]:
for i, j in zip(pred, y):
    print(i, j)

In [ ]:
'HANDY AND : YOU CAN USE IT ALMOST ANYWHERE'.split()

In [ ]:
from functions import *
import numpy as np
import pandas as pd
import pickle
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.manifold import TSNE

from keras import backend as K
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.metrics import Precision, Recall, AUC
from keras.optimizers import RMSprop, Adam

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #get rid of warnings



def create_y_matrix(y_data):
  y_matrix = np.zeros((len(y_data),2))
  for count,i in enumerate(y_data):
    if i == 1:
      y_matrix[count][1] = 1.0
    else:
      y_matrix[count][0] = 1.0
  return y_matrix

def create_X_matrix(dataset, w2v,word2vec_len=300, batch_size=25):
    dataset_size = len(dataset)
    x_matrix = np.zeros((dataset_size, batch_size, word2vec_len))    
    for i, line in enumerate(dataset):
        # print(line)
        # print(i)
        try:
            words = line.split()
            words = words[:batch_size] #cut off if too long
            for j, word in enumerate(words):
                if word in w2v:
                    x_matrix[i, j, :] = w2v[word]
        except:
            pass
    return x_matrix


def build_model(batch_size=25, word2vec_len=300):
    model = None
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(batch_size, word2vec_len)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))

    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))              
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',Precision(),Recall(),AUC()] )
    return model

def evaluate_model(model, X_test, y_test):
    eval = model.evaluate(X_test, y_test)
    loss = eval[0]
    accuracy = eval[1]
    precision = eval[2]
    recall = eval[3]
    auc = eval[4]    
    f1_score = (2*precision*recall)/(precision+recall)
    # print("Loss: ", loss)
    # print("Accuracy: ", accuracy)
    # print("Precision: ", precision)
    # print("Recall: ", recall)
    # print("AUC: ", auc)
    # print ('f1 score: ', f1_score)
    return loss, accuracy, precision, recall, auc, f1_score


def run(dataset_name,aug_method,n_sample):
    # Load data
    print('Loading data...')
    path_train_original = f'data/{dataset_name}/train.txt'
    path_train_aug = f'data/{dataset_name}/train_{dataset_name}_{aug_method}_n_sample_{n_sample}.csv'
    path_test = f'data/{dataset_name}/test.txt'
    train_aug = load_data(path_train_aug)
    train_original = load_data(path_train_original)
    test_data = load_data(path_test)
    X_train_aug, y_train_aug = train_aug['text'].values, train_aug['class'].values.astype(float)
    X_train_original, y_train_original = train_original['text'].values, train_original['class'].values.astype(float)
    X_test, y_test = test_data['text'].values, test_data['class'].values.astype(float)

    # y_train_aug = to_categorical(y_train_aug)
    # y_train_original = to_categorical(y_train_original)
    # y_test = to_categorical(y_test)


    # load wor2vec pickle
    path_w2v = f'data/{dataset_name}/word2vec.p'
    w2v = pickle.load(open(path_w2v, 'rb'))


    # create matrices
    print('Creating matrices...')
    X_train_aug = create_X_matrix(X_train_aug, w2v)
    X_train_original = create_X_matrix(X_train_original, w2v)
    X_test = create_X_matrix(X_test, w2v)

    # Train model
    print('Training model...')
    model_aug = build_model()
    model_original = build_model()

    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

    model_aug.fit(X_train_aug, y_train_aug, epochs=1000,
                 callbacks=callbacks, validation_split=0.1,
                  batch_size=1024,shuffle=True, verbose=0)
    model_original.fit(X_train_original, y_train_original, epochs=1000,
                    callbacks=callbacks, validation_split=0.1,
                    batch_size=1024,shuffle=True, verbose=0)

    # Evaluate model
    print('Evaluating model...')

    loss_org, accuracy_org, precision_org, recall_org, auc_org, f1_score_org = evaluate_model(model_original, X_test, y_test)
    loss_aug, accuracy_aug, precision_aug, recall_aug, auc_aug, f1_score_aug = evaluate_model(model_aug, X_test, y_test)
    print(f'original model: \n acc: {accuracy_org:.4f} \n f1: {f1_score_org:.4f} \n precision: {precision_org:.4f} \n recall: {recall_org:.4f}, \n auc: {auc_org:.4f}')
    print(f'augmented model: \n acc: {accuracy_aug:.4f} \n f1: {f1_score_aug:.4f} \n precision: {precision_aug:.4f} \n recall: {recall_aug:.4f}, \n auc: {auc_aug:.4f}')

    print('finished')



run('pc','aeda_augmenter',4)

In [ ]:
from functions import *
import numpy as np
import pandas as pd
import pickle
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.manifold import TSNE

from keras import backend as K
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.metrics import Precision, Recall, AUC
from keras.optimizers import RMSprop, Adam
import runai.ga.keras


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #get rid of warnings



def create_y_matrix(y_data):
  y_matrix = np.zeros((len(y_data),2))
  for count,i in enumerate(y_data):
    if i == 1:
      y_matrix[count][1] = 1.0
    else:
      y_matrix[count][0] = 1.0
  return y_matrix

def create_X_matrix(dataset, w2v,word2vec_len=300, batch_size=25):
    dataset_size = len(dataset)
    x_matrix = np.zeros((dataset_size, batch_size, word2vec_len))    
    for i, line in enumerate(dataset):
        try:
            words = line.split()
            words = words[:batch_size] #cut off if too long
            for j, word in enumerate(words):
                if word in w2v:
                    x_matrix[i, j, :] = w2v[word]
        except:
            pass
    return x_matrix


def build_model(batch_size=25, word2vec_len=300):
    model = None
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(batch_size, word2vec_len)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))

    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))  

    STEPS = 10000
    optimizer = runai.ga.keras.optimizers.Optimizer(Adam(), steps=STEPS)            
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc',Precision(),Recall(),AUC()] )
    return model

def evaluate_model(model, X_test, y_test):
    eval = model.evaluate(X_test, y_test)
    loss = eval[0]
    accuracy = eval[1]
    precision = eval[2]
    recall = eval[3]
    auc = eval[4]    
    f1_score = (2*precision*recall)/(precision+recall)
    # print("Loss: ", loss)
    # print("Accuracy: ", accuracy)
    # print("Precision: ", precision)
    # print("Recall: ", recall)
    # print("AUC: ", auc)
    # print ('f1 score: ', f1_score)
    return loss, accuracy, precision, recall, auc, f1_score


def run(dataset_name,aug_method,n_sample):
    # Load data
    print('Loading data...')
    path_train_original = f'data/{dataset_name}/train.txt'
    path_train_aug = f'data/{dataset_name}/train_{dataset_name}_{aug_method}_n_sample_{n_sample}.csv'
    path_test = f'data/{dataset_name}/test.txt'
    train_aug = load_data(path_train_aug)
    train_original = load_data(path_train_original)
    test_data = load_data(path_test)
    X_train_aug, y_train_aug = train_aug['text'].values, train_aug['class'].values.astype(float)
    X_train_original, y_train_original = train_original['text'].values, train_original['class'].values.astype(float)
    X_test, y_test = test_data['text'].values, test_data['class'].values.astype(float)

    # y_train_aug = to_categorical(y_train_aug)
    # y_train_original = to_categorical(y_train_original)
    # y_test = to_categorical(y_test)


    # load wor2vec pickle
    path_w2v = f'data/{dataset_name}/word2vec.p'
    w2v = pickle.load(open(path_w2v, 'rb'))


    # create matrices
    print('Creating matrices...')
    X_train_aug = create_X_matrix(X_train_aug, w2v)
    X_train_original = create_X_matrix(X_train_original, w2v)
    X_test = create_X_matrix(X_test, w2v)

    # Train model
    print('Training model...')
    model_aug = build_model()
    model_original = build_model()

    callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

    model_aug.fit(X_train_aug, y_train_aug, epochs=1000,
                 callbacks=callbacks, validation_split=0.1,
                  batch_size=128,shuffle=True, verbose=0)
    model_original.fit(X_train_original, y_train_original, epochs=1000,
                    callbacks=callbacks, validation_split=0.1,
                    batch_size=128,shuffle=True, verbose=0)

    # Evaluate model
    print('Evaluating model...')

    loss_org, accuracy_org, precision_org, recall_org, auc_org, f1_score_org = evaluate_model(model_original, X_test, y_test)
    loss_aug, accuracy_aug, precision_aug, recall_aug, auc_aug, f1_score_aug = evaluate_model(model_aug, X_test, y_test)
    print(f'original model: \n acc: {accuracy_org:.4f} \n f1: {f1_score_org:.4f} \n precision: {precision_org:.4f} \n recall: {recall_org:.4f}, \n auc: {auc_org:.4f}')
    print(f'augmented model: \n acc: {accuracy_aug:.4f} \n f1: {f1_score_aug:.4f} \n precision: {precision_aug:.4f} \n recall: {recall_aug:.4f}, \n auc: {auc_aug:.4f}')

    print('finished')

In [ ]:
run('pc','aeda_augmenter',8)

In [ ]:
1024//100

## compare results

In [ ]:
pc_aeda_large = pd.read_csv('results/pc_aeda_compare_with_large_w2v.csv', index_col=0)
pc_aeda_small = pd.read_csv('results/pc_aeda_compare_with_small_w2v.csv', index_col=0)
pc_eda_large = pd.read_csv('results/pc_eda_compare_with_large_w2v.csv', index_col=0)
pc_eda_small = pd.read_csv('results/pc_eda_compare_with_small_w2v.csv', index_col=0)
pc_wordnet_large = pd.read_csv('results/pc_wordnet_compare_with_large_w2v.csv', index_col=0)
pc_wordnet_small = pd.read_csv('results/pc_wordnet_compare_with_small_w2v.csv', index_col=0)

In [ ]:
list_of_results = [pc_aeda_large,pc_aeda_small,pc_eda_large ,pc_eda_small ,pc_wordnet_large,pc_wordnet_small]
list_of_features = ['loss','accuracy','precision','recall','auc''f1_score']


In [ ]:
df = pd.concat(list_of_results)
df = df.groupby(df.index,sort=False).mean()
org_series = df['original']

In [ ]:
for i in list_of_results:
    i['original'] = org_series

In [ ]:
for i in list_of_results:
    i.to_csv(f'results/{i.index[0]}.csv')

In [4]:
pc_aeda_large.to_excel('results/pc_aeda_large_w2v.xlsx')
pc_aeda_small.to_excel('results/pc_aeda_small_w2v.xlsx')
pc_eda_large.to_excel('results/pc_eda_large_w2v.xlsx')
pc_eda_small.to_excel('results/pc_eda_small_w2v.xlsx')
pc_wordnet_large.to_excel('results/pc_wordnet_large_w2v.xlsx')
pc_wordnet_small.to_excel('results/pc_wordnet_small_w2v.xlsx')

In [ ]:
pc_eda_small

In [ ]:
df = pd.read_csv('results/pc_aeda_large_w2v.csv', index_col=0)

In [ ]:
df

In [ ]:
df = pd.read_csv('results/pc_aeda_small_w2v.csv', index_col=0)

In [ ]:
df

In [3]:
pc_aeda_large = pd.read_csv('results/pc_aeda_large_w2v.csv', index_col=0)
pc_aeda_small = pd.read_csv('results/pc_aeda_small_w2v.csv', index_col=0)
pc_eda_large = pd.read_csv('results/pc_eda_large_w2v.csv', index_col=0)
pc_eda_small = pd.read_csv('results/pc_eda_small_w2v.csv', index_col=0)
pc_wordnet_large = pd.read_csv('results/pc_wordnet_large_w2v.csv', index_col=0)
pc_wordnet_small = pd.read_csv('results/pc_wordnet_small_w2v.csv', index_col=0)

In [ ]:
print(pc_aeda_large.to_markdown())

In [ ]:
print(pc_aeda_small.to_markdown())

In [ ]:
print(pc_eda_large.to_markdown())

In [ ]:
print(pc_eda_small.to_markdown())

In [ ]:
print(pc_wordnet_large.to_markdown())

In [ ]:
print(pc_wordnet_small.to_markdown())